In [1]:
!pip install tensorflow keras opencv-python matplotlib streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.4 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()  # select kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ntsejuty","key":"d0bd591a3fc702f6b2832836fe86dab1"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download PlantVillage dataset
!kaggle datasets download -d emmarex/plantdisease

# Unzip dataset
!unzip -q plantdisease.zip -d plant_disease_data

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 97% 637M/658M [00:03<00:00, 191MB/s]
100% 658M/658M [00:05<00:00, 133MB/s]


In [5]:
import os

data_dir = "plant_disease_data/PlantVillage"
# Get all class names (subfolders)
all_classes = sorted(os.listdir(data_dir))

print("Total classes:", len(all_classes))
print("Sample classes:", all_classes)

Total classes: 15
Sample classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


# Image Preprocessing

In [7]:
img_size = (128, 128)   # resize all images to 128x128
batch_size = 32

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


## Training Data Loader

In [9]:
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)
train_gen

Found 16516 images belonging to 15 classes.


## Validation Data Loader

In [10]:
val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)
val_gen

Found 4122 images belonging to 15 classes.


## Get Class Labels

In [11]:
class_names = list(train_gen.class_indices.keys())
print("Classes:", class_names)
print("Total Classes:", len(class_names))

Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Total Classes: 15


# Build Model

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


In [13]:
# Load Pretrained Base Model
base_model = MobileNetV2(weights="imagenet", include_top=False,
                         input_shape=(128, 128, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
# Freezes pretrained weights (don’t update during training).
base_model.trainable = False

In [15]:
#add custom layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(len(class_names), activation="softmax")(x)


In [16]:
# Final model
model = Model(inputs=base_model.input, outputs=preds)

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Train Model

In [17]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


517/517 ━━━━━━━━━━━━━━━━━━━━ 254s 480ms/step - accuracy: 0.5858 - loss: 1.3452 - val_accuracy: 0.8455 - val_loss: 0.4817
Epoch 2/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 287s 555ms/step - accuracy: 0.8374 - loss: 0.4872 - val_accuracy: 0.8666 - val_loss: 0.4044
Epoch 3/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 253s 489ms/step - accuracy: 0.8676 - loss: 0.4091 - val_accuracy: 0.8802 - val_loss: 0.3634
Epoch 4/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 292s 565ms/step - accuracy: 0.8783 - loss: 0.3616 - val_accuracy: 0.8874 - val_loss: 0.3394
Epoch 5/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 285s 550ms/step - accuracy: 0.8846 - loss: 0.3339 - val_accuracy: 0.8976 - val_loss: 0.3081
Epoch 6/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 262s 507ms/step - accuracy: 0.8957 - loss: 0.3109 - val_accuracy: 0.8836 - val_loss: 0.3405
Epoch 7/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 286s 553ms/step - accuracy: 0.9020 - loss: 0.3006 - val_accuracy: 0.8998 - val_loss: 0.3026
Epoch 8/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 284s 549ms/step - accuracy: 0.8989 - loss: 0.29

In [18]:
#Save Model
model.save("plant_disease_model_15.h5")
print("Model saved as plant_disease_model_15.h5")

Model saved as plant_disease_model_15.h5


# Streamlit App

In [33]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

# Load trained model
model = tf.keras.models.load_model("plant_disease_model_15.h5")

# Class labels
class_names = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]

st.title("Plant Disease Detector (15 Classes)")
st.write("Be a doctor for your Pepper, Potato or Tomato plant!")

uploaded_file = st.file_uploader("Upload a leaf image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, 1)
    img_resized = cv2.resize(img, (128, 128))
    img_norm = img_resized / 255.0
    img_input = np.expand_dims(img_norm, axis=0)

    # Prediction
    preds = model.predict(img_input)
    label = class_names[np.argmax(preds)]
    confidence = np.max(preds) * 100

    st.image(img, channels="BGR", caption="Uploaded Image", use_container_width=True)
    st.success(f"Prediction: {label} ({confidence:.2f}%)")

Overwriting app.py


**Steps:**

1.  Go to the ngrok website (https://dashboard.ngrok.com/signup) and sign up for a free account.
2.  Once logged in, go to the "Your Authtoken" section (https://dashboard.ngrok.com/get-started/your-authtoken) to find your authtoken.
3.  In Colab, click on the "🔑" icon in the left sidebar to open the Secrets Manager.
4.  Click "Add new Secret".
5.  For the "Name" field, enter `NGROK_AUTH_TOKEN`.
6.  For the "Value" field, paste your authtoken from the ngrok website.
7.  Make sure the "Notebook access" toggle is enabled for this notebook.

After adding the secret, run the following cell to configure ngrok with your authtoken:

In [34]:
import os
NGROK_AUTH_TOKEN = "YOUR_API_KEY"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [36]:
# Install dependencies
!pip install streamlit pyngrok --quiet

# Kill any existing streamlit processes
!pkill -f streamlit

# Start Streamlit in the background
get_ipython().system_raw('streamlit run app.py &')

from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Open this URL in your browser:", public_url)

Open this URL in your browser: NgrokTunnel: "https://45ed8bd29e71.ngrok-free.app" -> "http://localhost:8501"
